In [1]:
%cd ..

/mnt/cup/labs/hasson/zaid/fmri-convs


In [2]:
import pandas as pd
from util.path import Path
from glob import glob

## Transcript QA

In [3]:
transpath = Path(root='stimuli', datatype='transcript', suffix='transcript', ext='.csv')
files = glob(transpath.starstr(['conv', 'datatype']))
len(files)

202

In [4]:
dfs = []
for filename in files:
    df = pd.read_csv(filename, index_col=0)
    filepath = Path.frompath(filename)
    df.insert(0, 'conv', filepath['conv'])
    dfs.append(df)
df = pd.concat(dfs)
df.sort_values(['conv', 'run', 'trial', 'onset'], inplace=True)
df.reset_index(drop=True, inplace=True)

print(len(df))
df.head()

98691


,conv,run,trial,speaker,turn,utterance,sentence_id,token,token_norm,onset,offset
0,101,1,1,101,1,0,0.0,Hi,hi,4.00,4.34
1,101,1,1,101,1,0,1.0,Um,um,4.34,4.59
2,101,1,1,101,1,0,1.0,so,so,4.72,5.18
3,101,1,1,101,1,0,1.0,given,given,5.36,5.69
4,101,1,1,101,1,0,1.0,the,the,5.69,7.92


In [10]:
# which conversations are missing transcripts?
tcount = df[['conv', 'trial']].drop_duplicates().groupby(['conv']).count()
tcount

,trial
conv,
101,10
104,10
105,10
106,10
107,10
112,10
114,10
120,10
123,10


In [28]:
tcount[tcount['trial'] == 10].indexA

Index([101, 104, 105, 106, 107, 112, 114, 120, 123, 126, 157], dtype='int64', name='conv')

In [25]:
# which trials are missing transcripts?
tdf = df[['conv', 'run', 'trial']].drop_duplicates()
tdf[tdf.conv.isin(tcount[(tcount['trial'] < 10)].index.values)]

,conv,run,trial
50517,128,1,1
50873,128,1,4
51312,128,2,8
51790,128,3,9
52233,128,3,11
...,...,...,...
96214,174,3,12
96677,174,4,13
97198,174,4,15
97683,174,5,18


In [6]:
# Number of words per trial
df.groupby(['conv', 'run', 'trial']).token.count().describe()

count    191.000000
mean     488.596859
std       55.491866
min      285.000000
25%      462.500000
50%      493.000000
75%      521.500000
max      665.000000
Name: token, dtype: float64

In [7]:
# Number of words per trial per speaker (how short turns are between speakers)
df.groupby(['conv', 'run', 'trial', 'speaker']).token.count().describe()

count    382.000000
mean     244.298429
std       62.107222
min       58.000000
25%      205.000000
50%      244.500000
75%      280.000000
max      469.000000
Name: token, dtype: float64

In [33]:
# Number of words per conversations
df.conv.value_counts(sort=False)

conv
101    4958
104    4351
105    4546
106    4734
107    5456
112    4544
114    4721
120    3945
123    3596
126    5291
128    4003
131    3154
132     995
133    3573
137    2515
142    3408
153    4568
156    3129
157    4502
158    2684
163    3920
171    3584
173    3683
174    3462
Name: count, dtype: int64

In [47]:
df.conv.value_counts().describe()

count      24.000000
mean     3888.416667
std       988.321674
min       995.000000
25%      3448.500000
50%      3932.500000
75%      4551.500000
max      5456.000000
Name: count, dtype: float64

In [42]:
# Runs per conversation
# Should be 5 runs per conversation
df[['conv', 'run']].drop_duplicates().reset_index(drop=True).groupby('conv').run.count()

conv
101    5
104    5
105    5
106    5
107    5
112    5
114    5
120    5
123    5
126    5
128    5
131    4
132    2
133    5
137    4
142    5
153    5
156    4
157    5
158    4
163    5
171    4
173    5
174    4
Name: run, dtype: int64

In [7]:
df[['conv', 'trial']].drop_duplicates().reset_index(drop=True).groupby('conv').trial.count()

conv
101    10
104     9
105     9
106    10
107    10
112     9
114     9
120     8
123     8
126     9
128     9
131     7
132     2
133     9
137     5
142     7
153     9
156     6
157     9
158     6
163     8
171     7
173     9
174     7
Name: trial, dtype: int64

In [6]:
# Trials per conversation
# Should be 20 trials per conversation, or 4 per run, 2 generate, 2 read
with pd.option_context('display.max_rows', None):
    print(df[['conv', 'run', 'trial']].drop_duplicates().reset_index(drop=True).groupby(['conv', 'run']).trial.count())

conv  run
101   1      2
      2      2
      3      2
      4      2
      5      2
104   1      2
      2      2
      3      2
      4      2
      5      2
105   1      2
      2      2
      3      2
      4      2
      5      2
106   1      2
      2      2
      3      2
      4      2
      5      2
107   1      2
      2      2
      3      2
      4      2
      5      2
112   1      2
      2      2
      3      2
      4      2
      5      2
114   1      2
      2      2
      3      2
      4      2
      5      2
120   1      2
      2      2
      3      2
      4      2
      5      2
123   1      2
      2      2
      3      2
      4      2
      5      2
126   1      2
      2      2
      3      2
      4      2
      5      2
128   1      2
      2      1
      3      2
      4      2
      5      2
131   1      2
      2      2
      3      2
      4      2
132   1      1
      4      1
133   1      1
      2      2
      3      2
      4      2
      5      2


In [35]:
df.groupby(['conv', 'run']).run.count()

conv  run
101   1       943
      2       947
      3       999
      4      1029
      5      1040
             ... 
173   5       405
174   1       468
      3       980
      4      1006
      5      1008
Name: run, Length: 111, dtype: int64